## Mover extensiones a un disco nuevo

Debemos estar en la máquina virtual con los 5 discos sin formato (si sus discos duros tienen formato, favor restaurar la instantánea apropiada)

Logueados como el usuario **root**, nos aseguramos de que tengamos los discos sin formato

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
```
***

Ahora revisamos el script para asegurarse que se formatearan los 5 discos, pero agregaremos otra instrucción al script para crear un volúmen de grupo como se muestra a continuación

`cp 123 12345`

`nano 12345`

y agregar lo siguiente

***
```
# crear un volumen de grupo usando 4 volumenes fisicos
vgcreate newvg /dev/sdb1 /dev/sdc1 /dev/sdd1 /dev/sde1

#crear 4 volumenes de grupo
lvcreate -n newlv1 newvg -L 3G
lvcreate -n newlv2 newvg -L 4G
lvcreate -n newlv3 newvg -L 5G
lvcreate -n newlv4 newvg -L 6G
```
***

quedando el contenido del archivo de la siguiente forma

**12345**

***
```
#!/bin/bash
#Estos comandos Establecen que van a ser discos msdos master boot record 
parted /dev/sdb mklabel msdos
parted /dev/sdc mklabel msdos
parted /dev/sdd mklabel msdos
parted /dev/sde mklabel msdos
parted /dev/sdf mklabel msdos

#Estos comandos crean una particion primaria y le hacen un set a lvm
parted -s /dev/sdb unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdc unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdd unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sde unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdf unit mib mkpart primary 1 100% set 1 lvm on

# Solo si es RHEL8
#Este comando le dice al kernel que reconozca esto nuevo
#udevadm settle

# De lo contrario ejecute
partprobe -s

# crear un volumen de grupo usando 4 volumenes fisicos
vgcreate newvg /dev/sdb1 /dev/sdc1 /dev/sdd1 /dev/sde1

#crear 4 volumenes de grupo
lvcreate -n newlv1 newvg -L 3G
lvcreate -n newlv2 newvg -L 4G
lvcreate -n newlv3 newvg -L 5G
lvcreate -n newlv4 newvg -L 6G

```
***

Las ultimas cuatro lineas crean cuatro volumenes lógicos de tamaño 3GB, 4GB, 5GB y 6 GB para un total de 18 GB.

Ahora procedemos a ejecutar el script **12345**

`sh 12345`

y su salida es 

***
```
root@debian12lvm1:~# sh 12345
Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

/dev/sda: msdos partitions 1 2 <5>                                        
/dev/sdb: msdos partitions 1
/dev/sdc: msdos partitions 1
/dev/sdd: msdos partitions 1
/dev/sde: msdos partitions 1
/dev/sdf: msdos partitions 1
  Physical volume "/dev/sdb1" successfully created.
  Physical volume "/dev/sdc1" successfully created.
  Physical volume "/dev/sdd1" successfully created.
  Physical volume "/dev/sde1" successfully created.
  Volume group "newvg" successfully created
  Logical volume "newlv1" created.
  Logical volume "newlv2" created.
  Logical volume "newlv3" created.
  Logical volume "newlv4" created.
```
***

Ejecutamos el siguiente comando para saber cómo tenemos distribuido el espacio de los volúmenes

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree    Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g 1012.00m <4.01g
  /dev/sdc1  newvg lvm2 a--  <5.00g 1020.00m  4.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g       0  <5.00g

```
***

en **/dev/sdb1** tenemos casi un GB libre y lo mismo sucede en **dev/sdc1**, entonces, lo que haremos es reemplazar un disco

Lo primero que hay que hacer es preparar el disco que hay que agregar el volumen al grupo

`vgextend newvg /dev/sdf1`

***
```
root@debian12lvm1:~# vgextend newvg /dev/sdf1
  Physical volume "/dev/sdf1" successfully created.
  Volume group "newvg" successfully extended

```
***

Ejecutamos el siguiente comando para saber cómo tenemos distribuido el espacio de los volúmenes

`pvs -o+pv_used`

***
```
  PV         VG    Fmt  Attr PSize  PFree    Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g 1012.00m <4.01g
  /dev/sdc1  newvg lvm2 a--  <5.00g 1020.00m  4.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sdf1  newvg lvm2 a--  <5.00g   <5.00g     0
```
***

Podemos observar el espacio libre que esta en **/dev/sdf1** y ahora procedemos a mover os extends del disco que deseamos eliminar hacia el nuevo disco

`pvmove /dev/sdb1 /dev/sdf1`


***
```
root@debian12lvm1:~# pvmove /dev/sdb1 /dev/sdf1
  /dev/sdb1: Moved: 0.58%
  /dev/sdb1: Moved: 74.85%
  /dev/sdb1: Moved: 74.95%
  /dev/sdb1: Moved: 100.00%

```
***

Primero debemos verificar que todo este bien usando el siguiente comando
`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree    Used  
  /dev/sdb1  newvg lvm2 a--  <5.00g   <5.00g     0 
  /dev/sdc1  newvg lvm2 a--  <5.00g 1020.00m  4.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sdf1  newvg lvm2 a--  <5.00g 1012.00m <4.01g

```
***

Ahora debemos sacar ese volumen físico dañado de nuestro volúmen de grupo

`vgreduce newvg /dev/sdb1`

***
```
root@debian12lvm1:~# vgreduce newvg /dev/sdb1
  Removed "/dev/sdb1" from volume group "newvg"

```
***

Y ahora ejecutamos el comando

`pvs -o+pv_used`

***
```
root@debian12lvm1:~# pvs -o+pv_used
  PV         VG    Fmt  Attr PSize  PFree    Used  
  /dev/sdb1        lvm2 ---  <5.00g   <5.00g     0 
  /dev/sdc1  newvg lvm2 a--  <5.00g 1020.00m  4.00g
  /dev/sdd1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sde1  newvg lvm2 a--  <5.00g       0  <5.00g
  /dev/sdf1  newvg lvm2 a--  <5.00g 1012.00m <4.01g
```
***

Podemos observar que **/dev/sdb1** ya no forma parte del volumen de grupo y ahora si podemos quitar el disco físicamente o reemplazarlo.


